<a href="https://colab.research.google.com/github/ran4erep/Stable-Colab/blob/main/Stable_Diffusion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Установка Stable Diffusion и подключение Google Drive: { vertical-output: true, form-width: "100%", display-mode: "form" }
# @markdown Хотите исользовать Google Drive?
use_gdrive = False # @param {type:"boolean"}
import os
import torch
from google.colab import files
from PIL import Image
import numpy as np
from datetime import date
from datetime import datetime

if use_gdrive:
  from google.colab import drive
  drive.mount('/content/gdrive')
  print("GDrive sucessfully mounted to /content/gdrive")
  # Создаём папку в Google Drive, если её нет
  if not os.path.exists("/content/gdrive/MyDrive/SDOutput") and use_gdrive:
    os.makedirs("/content/gdrive/MyDrive/SDOutput", exist_ok=True)
    print("Успешно создана папка /content/gdrive/MyDrive/SDOutput")
else:
  print("Google Drive не подключен")

!pip install diffusers["torch"] transformers
!pip install accelerate
!pip install git+https://github.com/huggingface/diffusers
!pip install compel --upgrade

#from diffusers import StableDiffusionPipeline
from diffusers import AutoPipelineForText2Image
from diffusers import (
    DDPMScheduler,
    DDIMScheduler,
    PNDMScheduler,
    LMSDiscreteScheduler,
    EulerAncestralDiscreteScheduler,
    EulerDiscreteScheduler,
    DPMSolverMultistepScheduler,
)
from diffusers.utils import make_image_grid
from compel import Compel, ReturnedEmbeddingsType

In [ ]:
# @title Подготовка Stable Diffusion: { vertical-output: true, form-width: "100%", display-mode: "form" }
# @markdown
# @markdown В переменной **current_checkpoint** указываем чекпоинт модель с HuggingFace,  взять её можно на [CivitAI](https://civitai.com/models). При поиске моделей, в фильтре, ставим галочку только на checkpoint.
# @markdown
# @markdown  Потом уже заходим на [HuggingFace](https://huggingface.co/models?library=diffusers) и ищем там понравившеюся модель по её названию с CivitAI. Копируем это название и вставляем в значение переменной **current_checkpoint**.

!nvidia-smi

####################
# Список опробованных мною моделей:
# PicX_real:       GraydientPlatformAPI/picx-real
# Juggernaut:      digiplay/Juggernaut_final
# Juggernaut XL:   frankjoshua/juggernautXL_version6Rundiffusion
# RealVisXL 3.0:   SG161222/RealVisXL_V3.0_Turbo
# Base 1.5:        runwayml/stable-diffusion-v1-5
# Base SDXL:       stabilityai/stable-diffusion-xl-base-1.0
# Base SDXL Turbo: stabilityai/sdxl-turbo
####################

# @markdown Текущая модель:
current_checkpoint = "SG161222/RealVisXL_V3.0_Turbo" # @param ["runwayml/stable-diffusion-v1-5", "stabilityai/stable-diffusion-xl-base-1.0", "stabilityai/sdxl-turbo", "digiplay/Juggernaut_final", "GraydientPlatformAPI/picx-real", "frankjoshua/juggernautXL_version6Rundiffusion", "SG161222/RealVisXL_V3.0_Turbo", "SG161222/Realistic_Vision_V6.0_B1_noVAE"] {allow-input: true}
# @markdown Использовать LoRA?
use_lora = False # @param {type:"boolean"}
# @markdown Текущая LoRA:
lora_path = "nerijs/pixel-art-xl" # @param ["nerijs/pixel-art-xl"] {allow-input: true}
# @markdown Имя файла LoRA:
weight_name = "pixel-art-xl.safetensors" # @param {type:"string"}
# @markdown Текущее устройство:
current_device = "cuda" # @param ["cuda"]
## @markdown Вариант:
#current_variant = "fp16" # @param ["", "fp16", "ema"]
# @markdown Использовать safetensors?
use_safetensors = None # @param ["True", "False", "None"] {type:"raw"}

# Список планировщиков
ddpm = DDPMScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
ddim = DDIMScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
pndm = PNDMScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
lms = LMSDiscreteScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
euler_anc = EulerAncestralDiscreteScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
euler = EulerDiscreteScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
dpm = DPMSolverMultistepScheduler.from_pretrained(current_checkpoint, subfolder="scheduler")
# @markdown Текущий планировщик:
current_scheduler = euler_anc # @param ["ddpm", "ddim", "pndm", "lms", "euler_anc", "euler", "dpm"] {type:"raw"}

# Настройка пайплайна
pipe = AutoPipelineForText2Image.from_pretrained(current_checkpoint, torch_dtype=torch.float16, use_safetensors=use_safetensors)
pipe = pipe.to(current_device)
pipe.safety_checker = None # Отключение NSFW (18+)
pipe.scheduler = current_scheduler.from_config(pipe.scheduler.config) # <-- Здесь выбираем планировщик, список в переменных выше
if use_lora:
  pipe.load_lora_weights(lora_path, weight_name=weight_name)

In [ ]:
# @title Калькулятор соотношения сторон: { vertical-output: true, form-width: "100%", display-mode: "form" }
# @markdown Помогает определиться с высотой желаемого соотношения сторон
# @markdown
# @markdown Исходная ширина
calc_width = 1920 # @param {type:"integer"}
# @markdown Исходная высота
calc_height = 1080 # @param {type:"integer"}
# @markdown Новая ширина
new_calc_width = 512 # @param {type:"integer"}
answer = (calc_height / calc_width) * new_calc_width

answer

In [ ]:
# @title Настройка пайплайна: { run: "auto", form-width: "100%", display-mode: "form" }
# Здесь указываем переменные для пайплайна Stable Diffusion
# Дефолтный промпт для лучшего качества: (8k, ultra realistic, highly detailed, cinematic lighting)

# @markdown Промпт:
prompt = "RAW photo, fox in the forest, 8k, ultra realistic, highly detailed, cinematic lighting" # @param {type:"string"}
# @markdown Негативный промпт:
negs = "canvas frame, cartoon, 3d, disfigured, bad art, deformed, extra limbs, close up, b&w, wierd colors, blurry, duplicate, morbid, mutilated, out of frame, extra fingers, mutated hands, poorly drawn hands, poorly drawn face, mutation, deformed, ugly, blurry, bad anatomy, bad proportions, extra limbs, cloned face, disfigured, out of frame, ugly, extra limbs, bad anatomy, gross proportions, malformed limbs, missing arms, missing legs, extra arms, extra legs, mutated hands, fused fingers, too many fingers, long neck, Photoshop, video game, ugly, tiling, poorly drawn hands, poorly drawn feet, poorly drawn face, out of frame, mutation, mutated, extra limbs, extra legs, extra arms, disfigured, deformed, cross-eye, body out of frame, blurry, bad art, bad anatomy, 3d render" # @param {type:"string"}
# @markdown Для какой версии Stable Diffusion вы загрузили модель? (Это важно для работоспособности веса токенов, иначе будет ошибка)
sd_type = "Stable Diffusion XL (Turbo)" # @param ["Stable Diffusion 1.5/2.0", "Stable Diffusion XL (Turbo)"]
# @markdown Вес для SD 1.5 записывается как + или - после слова внутри токена: "cat playing with ball+++++", "cat+ playing with ball"
# @markdown
# @markdown Вес для SDXL записывается как дробное число после скобок, внутри которых есть слово или даже целый токен: "cat (playing with ball)1.5"
# @markdown
# @markdown Ширина:
width = 512 # @param {type:"number"}
# @markdown Высота:
height = 512 # @param {type:"number"}
# @markdown Количество изображений:
images_count = 4 # @param {type:"slider", min:1, max:10, step:1}
# @markdown Шаги:
steps = 25 # @param {type:"number"}
# @markdown Шкала инструкции:
gscale = 7.5 # @param {type:"number"}
# @markdown Использовать случайный seed?
randomness = True # @param {type:"boolean"}
# @markdown Seed:
seed = 2965321946159901 # @param {type:"integer"}

In [ ]:
# @title Работа Stable Diffusion { vertical-output: true, form-width: "100%", display-mode: "form" }
# Генерация картинки
# @markdown <-- Начать генерацию

if sd_type == "Stable Diffusion 1.5/2.0":
  compel_proc = Compel(tokenizer=pipe.tokenizer, text_encoder=pipe.text_encoder)
  prompt_embeds = compel_proc(prompt)

if sd_type == "Stable Diffusion XL (Turbo)":
  compel = Compel(
    tokenizer=[pipe.tokenizer, pipe.tokenizer_2] ,
    text_encoder=[pipe.text_encoder, pipe.text_encoder_2],
    returned_embeddings_type=ReturnedEmbeddingsType.PENULTIMATE_HIDDEN_STATES_NON_NORMALIZED,
    requires_pooled=[False, True]
  )
  conditioning, pooled = compel(prompt)

print("#############################################")
print("# Текущая модель Stable Diffusion: " + current_checkpoint)
if use_lora:
  print("# Текущая модель LoRA: " + lora_path)
print("# Промпт: " + prompt)
print("# Негативный промпт: " + negs)
print("# Разрешение: " + str(width) + "x" + str(height))
print("# Количество генерируемых изображений: " + str(images_count))

if not randomness:
  generator = torch.Generator(current_device).manual_seed(seed)
  print("# Seed этой генерации: " + str(seed) + "\n#############################################")
elif randomness:
  current_seed = torch.Generator(current_device).seed()
  generator = torch.Generator(current_device).manual_seed(current_seed)
  print("# Seed этой генерации: " + str(current_seed) + "\n#############################################")


#result = pipe(prompt=prompt, height=height, width=width, num_inference_steps=steps, guidance_scale=gscale, negative_prompt=negs, generator=generator, num_images_per_prompt=images_count)

if sd_type == "Stable Diffusion 1.5/2.0":
  result = pipe(prompt_embeds=prompt_embeds, height=height, width=width, num_inference_steps=steps, guidance_scale=gscale, negative_prompt=negs, generator=generator, num_images_per_prompt=images_count)

if sd_type == "Stable Diffusion XL (Turbo)":
  result = pipe(prompt_embeds=conditioning, pooled_prompt_embeds=pooled, height=height, width=width, num_inference_steps=steps, guidance_scale=gscale, negative_prompt=negs, generator=generator, num_images_per_prompt=images_count)

# Вывод и сохрарнение результата

if use_gdrive:
  today = str(date.today())
  if not os.path.exists("/content/gdrive/MyDrive/SDOutput/" + today):
    os.makedirs("/content/gdrive/MyDrive/SDOutput/" + today, exist_ok=True)
  for i in range(images_count):
    time = datetime.now()
    time = str(time.strftime("%H-%M-%S"))
    file_name = time + "_" + prompt + "_seed" + str(current_seed) + "_image" + str(i) +".png"
    file_name = "/content/gdrive/MyDrive/SDOutput/" + today + "/" + file_name
    images = make_image_grid(result.images, rows=1, cols=images_count)
    result.images[i].save(file_name)
    print("Сохранено: " + file_name)

make_image_grid(result.images, rows=1, cols=images_count)